In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

 
import scipy as sp 
import sklearn
import random 
import time 

from sklearn import preprocessing, model_selection


from sklearn.utils import shuffle

In [34]:
#reading data
data1=pd.read_csv("January 2019 new.csv")
data2=pd.read_csv("December.csv")
data3=pd.read_csv("October.csv")
data4=pd.read_csv("November.csv")


In [35]:
#concatenating all data into one dataframe
frames=[data1,data2,data3,data4]
data=pd.concat(frames)


In [36]:
#filling missing values
data=data.fillna(1)

In [49]:
#discretizing data for arrival delay
data['bins_dep_delay']=pd.cut(data['DEP_DELAY'],[-130,-7,-5,-2,0,10,30,2200],
                         labels=["Early by 30 mins-1 hr mins","early by 10 mins","early by 5 mins","On Time","early by 2","0-10 Mins","2-48 Hrs"])

In [50]:
data['bins_dep_delay'].value_counts()

early by 5 mins               516977
Early by 30 mins-1 hr mins    455976
early by 2                    341832
early by 10 mins              338133
On Time                       259569
2-48 Hrs                      246892
0-10 Mins                     220727
Name: bins_dep_delay, dtype: int64

In [51]:
data.isnull().sum()

DAY_OF_MONTH         0
DAY_OF_WEEK          0
OP_UNIQUE_CARRIER    0
OP_CARRIER_FL_NUM    0
ORIGIN               0
ORIGIN_CITY_NAME     0
ORIGIN_STATE_ABR     0
ORIGIN_STATE_NM      0
DEST                 0
DEST_CITY_NAME       0
DEST_STATE_ABR       0
DEST_STATE_NM        0
DEP_TIME             0
DEP_DELAY            0
ARR_TIME             0
ARR_DELAY            0
CANCELLED            0
DIVERTED             0
AIR_TIME             0
bins_dep_delay       0
dtype: int64

In [52]:
#removing rows that dont give much information
data=data.drop(['ARR_DELAY','CANCELLED','DIVERTED','AIR_TIME','DEP_DELAY'],axis=1)

In [53]:
data.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,bins_dep_delay
0,1,2,9E,5246,MSP,"Minneapolis, MN",MN,Minnesota,MKE,"Milwaukee, WI",WI,Wisconsin,1155.0,1259.0,2-48 Hrs
1,1,2,9E,5249,ATL,"Atlanta, GA",GA,Georgia,PHF,"Newport News/Williamsburg, VA",VA,Virginia,1002.0,1132.0,early by 2
2,1,2,9E,5249,PHF,"Newport News/Williamsburg, VA",VA,Virginia,ATL,"Atlanta, GA",GA,Georgia,1207.0,1408.0,early by 10 mins
3,1,2,9E,5250,MSP,"Minneapolis, MN",MN,Minnesota,CLE,"Cleveland, OH",OH,Ohio,1015.0,1257.0,early by 2
4,1,2,9E,5254,MSP,"Minneapolis, MN",MN,Minnesota,RDU,"Raleigh/Durham, NC",NC,North Carolina,1535.0,1854.0,early by 10 mins


In [54]:
#converting the data into arrays
data=data.values

carrier=LabelEncoder()
data[:,2]=carrier.fit_transform(data[:,2])
origin=LabelEncoder()
data[:,4]=origin.fit_transform(data[:,4])
origin_city=LabelEncoder()
data[:,5]=origin_city.fit_transform(data[:,5])
origin_state=LabelEncoder()
data[:,6]=origin_state.fit_transform(data[:,6])
dest=LabelEncoder()
data[:,7]=dest.fit_transform(data[:,7])
dest_city=LabelEncoder()
data[:,8]=dest_city.fit_transform(data[:,8])
dest_state=LabelEncoder()
data[:,9]=dest_state.fit_transform(data[:,9])

output=LabelEncoder()
data[:,-1]=output.fit_transform(data[:,-1])

In [55]:
#label encoding
carrier=LabelEncoder()
data[:,2]=carrier.fit_transform(data[:,2])
origin=LabelEncoder()
data[:,4]=origin.fit_transform(data[:,4])
origin_city=LabelEncoder()
data[:,5]=origin_city.fit_transform(data[:,5])
origin_state=LabelEncoder()
data[:,6]=origin_state.fit_transform(data[:,6])
origin_state_name=LabelEncoder()
data[:,7]=origin_state_name.fit_transform(data[:,7])
dest=LabelEncoder()
data[:,8]=dest.fit_transform(data[:,8])
dest_city=LabelEncoder()
data[:,9]=dest_city.fit_transform(data[:,9])
dest_state=LabelEncoder()
data[:,10]=dest_state.fit_transform(data[:,10])
dest_state_name=LabelEncoder()
data[:,11]=dest_state_name.fit_transform(data[:,11])
output=LabelEncoder()
data[:,-1]=output.fit_transform(data[:,-1])

In [56]:
data[:5,:]

array([[1, 2, 0, 5246, 227, 203, 21, 21, 215, 202, 49, 50, 1155.0,
        1259.0, 1],
       [1, 2, 0, 5249, 20, 18, 8, 8, 253, 225, 45, 47, 1002.0, 1132.0, 5],
       [1, 2, 0, 5249, 253, 225, 45, 47, 20, 18, 8, 8, 1207.0, 1408.0, 4],
       [1, 2, 0, 5250, 227, 203, 21, 21, 68, 67, 33, 33, 1015.0, 1257.0,
        5],
       [1, 2, 0, 5254, 227, 203, 21, 21, 277, 263, 25, 31, 1535.0,
        1854.0, 4]], dtype=object)

In [57]:
#split the data for training the model
from sklearn.model_selection import train_test_split

X=data[:,:-1]
y=data[:,-1]
y=y.astype(int)
y


array([1, 5, 4, ..., 4, 4, 2])

In [58]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=42,shuffle=True)
y_train.shape

(1190053,)

In [59]:
X_train.shape

(1190053, 14)

In [60]:
#creating a Logistic Regression model adn fitting training data
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
#predicting test data based on the model fit above
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.24


In [62]:
#reading new dataset to test the model
test=pd.read_csv("Full_Data.csv",parse_dates=['Date'])

In [63]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    #df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test)

In [64]:
test.head()

,Date,combo,OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,2020-01-12,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5919,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,536,1111,6,12
1,2020-02-04,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",1927,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,2249,631,1,4
2,2020-03-24,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5157,YX,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,552,543,1,24
3,2020-04-03,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",364,F9,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,1426,558,4,3
4,2020-05-01,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",3290,MQ,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,254,119,4,1


In [66]:
test=test.drop(['Date','combo'],axis=1)
#test=test.drop(['DEST_CITY_NAME','ORIGIN_STATE_NM'],axis=1)
test.head()

,OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,DAY_OF_WEEK,DAY_OF_MONTH
0,5919,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,536,1111,6,12
1,1927,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,2249,631,1,4
2,5157,YX,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,552,543,1,24
3,364,F9,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,1426,558,4,3
4,3290,MQ,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,254,119,4,1


In [68]:
test=test.dropna()
test=test.values

from sklearn.preprocessing import LabelEncoder

carrier=LabelEncoder()
test[:,1]=carrier.fit_transform(test[:,1])
origin=LabelEncoder()
test[:,2]=origin.fit_transform(test[:,2])
origin_city=LabelEncoder()
test[:,3]=origin_city.fit_transform(test[:,3])
origin_state=LabelEncoder()
test[:,4]=origin_state.fit_transform(test[:,4])
dest=LabelEncoder()
test[:,5]=dest.fit_transform(test[:,5])
dest_city=LabelEncoder()
test[:,6]=dest_city.fit_transform(test[:,6])
dest_state=LabelEncoder()
test[:,7]=dest_state.fit_transform(test[:,7])


In [69]:
from sklearn.preprocessing import LabelEncoder

carrier=LabelEncoder()
test[:,1]=carrier.fit_transform(test[:,1])
origin=LabelEncoder()
test[:,2]=origin.fit_transform(test[:,2])
origin_city=LabelEncoder()
test[:,3]=origin_city.fit_transform(test[:,3])
origin_state=LabelEncoder()
test[:,4]=origin_state.fit_transform(test[:,4])
origin_state_name=LabelEncoder()
test[:,5]=origin_state_name.fit_transform(test[:,5])
dest=LabelEncoder()
test[:,6]=dest.fit_transform(test[:,6])
dest_city=LabelEncoder()
test[:,7]=dest_city.fit_transform(test[:,7])
dest_state=LabelEncoder()
test[:,8]=dest_state.fit_transform(test[:,8])
dest_state_name=LabelEncoder()
test[:,9]=dest_state_name.fit_transform(test[:,9])

In [70]:
test_predictions = logreg.predict(test)

In [71]:
testoutput1=pd.DataFrame({'prediction':test_predictions})
testoutput1['prediction'].value_counts()

1    37966
0    16886
5     9403
2     1222
3      687
6      182
4       38
Name: prediction, dtype: int64

In [72]:
test1=pd.read_csv("Full_Data.csv",parse_dates=['Date'])
test1['output']=testoutput1['prediction']

In [73]:
def convert_date(df):
    df['DAY_OF_WEEK']=df['Date'].dt.dayofweek
    df['MONTH']=df['Date'].dt.month
    df['DAY_OF_MONTH']=df['Date'].dt.day
    #df['YEAR']=df['Date'].dt.year
    

convert_date(test1)

In [74]:
test1.head()

,Date,combo,OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEP_TIME,ARR_TIME,output,DAY_OF_WEEK,MONTH,DAY_OF_MONTH
0,2020-01-12,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5919,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,536,1111,0,6,1,12
1,2020-02-04,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",1927,EV,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,2249,631,1,1,2,4
2,2020-03-24,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",5157,YX,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,552,543,1,1,3,24
3,2020-04-03,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",364,F9,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,1426,558,1,4,4,3
4,2020-05-01,"MSP--Minneapolis, MN--MN--CVG--Cincinnati, OH--KY",3290,MQ,MSP,"Minneapolis, MN",MN,Minnesota,CVG,"Cincinnati, OH",KY,Kentucky,254,119,1,4,5,1


In [75]:
test1.to_csv("dep_delap_without_states.csv")

In [77]:
a=list(output.inverse_transform([0,1,2,3,4,5,6]))
b=list(output.classes_)
c=pd.DataFrame({'Class':a,
                'Label':b})
c


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,Class,Label
0,0-10 Mins,0-10 Mins
1,2-48 Hrs,2-48 Hrs
2,Early by 30 mins-1 hr mins,Early by 30 mins-1 hr mins
3,On Time,On Time
4,early by 10 mins,early by 10 mins
5,early by 2,early by 2
6,early by 5 mins,early by 5 mins
